In [1]:
from ctgan import load_demo
from ctgan import CTGAN
import pandas as pd

from sklearn.preprocessing import StandardScaler


In [2]:
df_m_0 = "min_0"
df_m_1 = "min_1"
df_m_2 = 'min_2'
# df_m_3 = pd.read_csv('../dataset/min_3.csv')
df_0 = 'informative_cluster_0'
df_1 = 'informative_cluster_1'
df_merged = 'merged_shuffled'
# df_2 = pd.read_csv('../dataset/informative_cluster_2.csv')
selected_file = df_merged
df= pd.read_csv("../dataset/" + selected_file + ".csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16413 entries, 0 to 16412
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   type            16413 non-null  int64  
 1   amount          16413 non-null  float64
 2   oldbalanceOrg   16413 non-null  float64
 3   newbalanceOrig  16413 non-null  float64
 4   oldbalanceDest  16413 non-null  float64
 5   newbalanceDest  16413 non-null  float64
dtypes: float64(5), int64(1)
memory usage: 769.5 KB


In [5]:
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])
df.head()

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
0,1,205480.23,5949.01,0.0,1235745.12,1441225.36
1,1,167806.93,167806.93,0.0,0.00,167806.93
2,1,181330.91,37778.00,0.0,36793.35,218124.26
3,4,167907.12,167907.12,0.0,0.00,0.00
4,4,1434440.31,1434440.31,0.0,0.00,0.00


In [6]:
from sdv.cag import SingleTableProgrammableConstraint

class TransactionBalanceLogic(SingleTableProgrammableConstraint):
    def is_valid(self, row):
        print(row)
        amount = row['amount']
        obo = row['oldbalanceOrg']
        nbo = row['newbalanceOrig']
        obd = row['oldbalanceDest']
        nbd = row['oldbalanceDest']

        # if any(v is None for v in [t, obo, nbo, obd]):
        #     return True  # skip NaNs
        return (obd >= 0) & (obo >= 0) & (nbo >= 0) & (amount>0) & (nbd >= 0)

    def transform(self, table_data):
        return table_data  # no transformation needed

    def get_updated_metadata(self, metadata):
        new_metadata = metadata.copy()
        return new_metadata

    def reverse_transform(self, table_data):
        return table_data


In [7]:
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import Metadata
metadata = Metadata.detect_from_dataframe(df)
synthesizer = CTGANSynthesizer(metadata)
synthesizer.add_constraints([TransactionBalanceLogic()])
synthesizer.fit(df)

I:\ETVOS\2nd semester\DS Lab 1\DSLAB-Project\venv1\Lib\site-packages\sdv\single_table\base.py:128: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


In [8]:
data = synthesizer.sample(df.shape[0],output_file_path="../dataset/fake_" + selected_file + ".csv")

       type     amount  oldbalanceOrg  newbalanceOrig  oldbalanceDest  \
0         1  249120.14     2377531.77          814.36        27632.75   
1         1   14785.81       42063.94            0.00       176827.96   
2         1       0.00           0.00         5009.06       331470.73   
3         3    2086.72       59208.48         3865.90         2404.70   
4         1       0.00      120641.34         9425.39      3402203.24   
...     ...        ...            ...             ...             ...   
16408     4   82951.18     1059501.43            0.00        20274.96   
16409     4  492669.66      252210.51        13056.84        38779.47   
16410     1  248086.73      222164.17          266.39       205416.25   
16411     3       0.00           0.00        11757.52            0.00   
16412     1  820169.91     1455161.11          717.63        24859.38   

       newbalanceDest  
0          4406226.70  
1          1122000.98  
2           352109.69  
3            30228.16  
4  